 Pydantic 這個套件，他主要是用來建立資料模型，拿來做資料的驗證與設定，可幫你驗證資料的 data type ，及是否符合規則 (像是對應欄位是否為 emil)
 透過 Pydantic 建立資模型，我們只需要 import 該套件下的 BaseModel 類別即可

## 操作方式和 DataClass 一樣
- BaseModel 是一個 class，讓開發者能夠透過繼承該類別並且利用 typing 註記類別屬性的型別，就能夠擁有基本的驗證功能。

In [2]:
from pydantic import BaseModel

class Person(BaseModel):   # Person繼承BaseModel
    # field 一定要有 type hint
    first_name:str
    last_name:str
    age:int
        
p = Person(first_name="James", last_name="Han", age=31)
print(p.first_name)
print(p.last_name)
print(p.age)
print(p)     #自己的repr   

James
Han
31
first_name='James' last_name='Han' age=31


## dataclass 做不到的
- 驗證資料

In [3]:
p1 = Person(first_name="James", last_name="Han", age="31")    #自動轉換型別
print(p1)                                    # 從字串轉成整數

first_name='James' last_name='Han' age=31


In [4]:
p2 = Person(first_name="James", last_name="Han", age="XXX")

ValidationError: 1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='XXX', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [5]:
try:
    p2 = Person(first_name="James", last_name="Han", age="XXX")
except Exception as error:
    print(error)     #驗證不過，保證資料的完整性

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='XXX', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


## 可以將純文字轉成pydantic實體
- model_validate_json()方法用於解析 JSON 數據，並以 Pydantic 模型物件的形式傳回經過驗證的資料。

In [6]:
# ''' 可製造 多行文字
# 純文字的json檔
data_json = '''
{
    "first_name":"James",
    "last_name":"Han",
    "age":"31"
}
'''
#用 model.validate 將字串變成實體 p3
p3 = Person.model_validate_json(data_json)
print(p3)

first_name='James' last_name='Han' age=31


### 過濾所需的資料 (當初定義class時，可選擇所需的欄位，並沒有height)

In [7]:
class Person(BaseModel):
    #field一定要有type hint
    first_name:str
    last_name:str
    age:int
    
data_json = '''
{
    "first_name":"john",
    "last_name":"Smith",
    "age":42,
    "height":173
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=42


### 可以設定取得的資料,如果沒有這個欄位,設定欄位預設值(default)

In [8]:
class Person(BaseModel):
    #field一定要有type hint
    first_name:str #一定要有資料
    last_name:str #一定要有資料
    age:int  

data_json = '''
{
    "first_name":"James",
    "last_name":"Han"    
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

ValidationError: 1 validation error for Person
age
  Field required [type=missing, input_value={'first_name': 'James', 'last_name': 'Han'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [9]:
class Person(BaseModel):
    #field一定要有type hint
    first_name:str #一定要有資料
    last_name:str  #一定要有資料
    age:int = 0    #有預設的值
    lucky_numbers:list[int] = []    #有預設的值

data_json = '''
{
    "first_name":"john",
    "last_name":"Smith"    
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=0 lucky_numbers=[]


In [10]:
class Person(BaseModel):
    #field一定要有type hint
    first_name:str #一定要有資料
    last_name:str #一定要有資料
    age:int = 0 #有預設的值
    lucky_numbers:list[int] = []

data_json = '''
{
    "first_name":"john",
    "last_name":"Smith",
    "lucky_numbers":[1,2,"3",4,5]    
}
'''
p4 = Person.model_validate_json(data_json)
print(p4)

first_name='john' last_name='Smith' age=0 lucky_numbers=[1, 2, 3, 4, 5]


In [11]:
data_json = '''
{
    "id":100,
    "First Name":"John",
    "LASTNAME":"Smith",
    "age in years": 42    
}
'''
# 欄位名稱有 空白 大小寫，但是變數名稱是不可有空白，此時需要import class: Field

from pydantic import BaseModel,Field

class Person(BaseModel):
    id_:int = Field(alias="id")
    first_name:str = Field(alias="First Name")
    last_name:str = Field(alias="LASTNAME")
    age:int = Field(alias="age in years",default=0)

p1 = Person.model_validate_json(data_json)
print(p1)

id_=100 first_name='John' last_name='Smith' age=42


Pydantic 的 Field 是一个用于定义模型字段的类，它允许你为模型中的每个字段指定类型、默认值、描述以及其他验证规则。以下是 Field 的一些常见用法：

1.定义字段类型：通过 Field 可以明确指定字段的数据类型，确保数据的类型安全。
2.设置默認值：可以在 Field 中为字段设置默認值，当创建模型实例时未提供该字段的值，将使用默认值。
3.添加描述信息：可以为字段添加描述信息，这有助于生成文档或在错误消息中提供更清晰的说明。
4.自定义验证：如果需要对字段进行额外的验证，可以在 Field 中使用 validator 参数来添加自定义的验证函数。
5.字段别名：有时候需要在不同的上下文中使用不同的字段名，可以通过 Field 的 alias 参数来实现。

6.控制序列化：通过 Field 的 exclude 参数，可以控制该字段是否被包含在模型的序列化输出中。
7.读取描述：可以使用 description 属性来获取字段的描述信息，这在自动生成API文档时非常有用。
8.额外信息：Field 还支持 title、example 等其他参数，用于提供更多关于字段的信息。
9.继承和共享：如果多个模型中有相同的字段定义，可以使用 Field 来实现字段的继承和共享，避免重复代码。
10.配合类型注解：Field 通常与 Python 的类型注解一起使用，以提供更强的类型检查和自动完成功能。

# 巢狀的Model

In [12]:
data:str = '''
{
    "firstName": "James",
    "lastName": "Han",
    "born":{
        "place":{
            "country":"Taiwan",
            "city": "Kaoshiung"
        },
        "date":"1992-11-01"
    }
}
'''
# 只要有Dictionary，由內而外層層建立BaseModel
from pydantic import BaseModel,Field
class Place(BaseModel):
    country:str
    city:str

class Born(BaseModel):
    place:Place
    date:str

class Person(BaseModel):
    first_name:str = Field(alias="firstName")
    last_name:str = Field(alias="lastName")
    born:Born

p1 = Person.model_validate_json(data)
print(p1.born.place.country)
print(p1.born.place.city)
print(p1.born.date)
print(p1.first_name)
print(p1.last_name)

Taiwan
Kaoshiung
1992-11-01
James
Han
